In [47]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
import torch
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [39]:
# Convert to DataFrame
df = pd.read_csv("../data/reviews.csv")

reviews = df['review'].tolist()
labels = df['voted_up'].tolist()

# Split data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

# Ensure all elements in train_texts and test_texts are strings
train_texts = [str(text) for text in train_texts]
test_texts = [str(text) for text in test_texts]

# Clean reviews to remove unusual characters
train_texts = [re.sub(r'[^\x00-\x7F]+', ' ', text) for text in train_texts]
test_texts = [re.sub(r'[^\x00-\x7F]+', ' ', text) for text in test_texts]

In [40]:
# Check if the device supports MPS (Metal Performance Shaders) for Apple Silicon or fallback to CPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Load BERT tokenizer and pre-trained model for sequence classification (binary classification)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

# Tokenize the training and test texts
train_encodings = tokenizer(
    train_texts, 
    truncation=True, 
    padding=True, 
    max_length=128, 
    return_tensors='pt'
)
test_encodings = tokenizer(
    test_texts, 
    truncation=True, 
    padding=True, 
    max_length=128, 
    return_tensors='pt'
)
# Convert encodings and labels to TensorDatasets
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_labels))
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], torch.tensor(test_labels))

# Create DataLoader for training and test sets
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=8)
test_dataloader = DataLoader(test_dataset, batch_size=8)

# Set up optimizer with weight decay
optimizer = AdamW(model.parameters(), lr=2e-5)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/homebrew/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [42]:
# # Training the BERT model for 3 epochs
# epochs = 3
# model.train()
# for epoch in range(epochs):
#     model.train()
#     total_loss = 0
#     for batch in train_dataloader:
#         # Unpack the batch and move tensors to device
#         batch_input_ids, batch_attention_mask, batch_labels = [x.to(device) for x in batch]

#         # Zero gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask, labels=batch_labels)

#         # Loss is automatically computed by the model when `labels` are provided
#         loss = outputs.loss

#         # Backward pass and optimization step
#         loss.backward()
#         optimizer.step()

#         total_loss += loss.item()

#     print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_dataloader):.4f}")

# # Save the fine-tuned model and tokenizer
# model.save_pretrained('../model/fine_tuned_bert_reviews')
# tokenizer.save_pretrained('../model/fine_tuned_bert_reviews')


Epoch 1/3, Loss: 0.2030
Epoch 2/3, Loss: 0.1156
Epoch 3/3, Loss: 0.0591


('../model/fine_tuned_bert_reviews/tokenizer_config.json',
 '../model/fine_tuned_bert_reviews/special_tokens_map.json',
 '../model/fine_tuned_bert_reviews/vocab.txt',
 '../model/fine_tuned_bert_reviews/added_tokens.json')

In [44]:
# Evaluate the model on the test set
model.eval()  # Set model to evaluation mode

# Store predictions and true labels
predictions, true_labels = [], []

# No need to compute gradients during evaluation
with torch.no_grad():
    for batch in test_dataloader:
        # Unpack the batch and move tensors to device
        batch_input_ids, batch_attention_mask, batch_labels = [x.to(device) for x in batch]

        # Forward pass
        outputs = model(input_ids=batch_input_ids, attention_mask=batch_attention_mask)
        logits = outputs.logits

        # Get predictions and store them
        preds = torch.argmax(logits, dim=-1).cpu().numpy()
        label_ids = batch_labels.cpu().numpy()

        # Append results for comparison
        predictions.extend(preds)
        true_labels.extend(label_ids)

Accuracy: 0.93
Precision: 0.95
Recall: 0.97
F1-Score: 0.96


In [48]:
# Calculate various metrics
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)
conf_matrix = confusion_matrix(true_labels, predictions)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.93
Precision: 0.95
Recall: 0.97
F1-Score: 0.96
Confusion Matrix:
[[ 836  288]
 [ 161 5067]]


In [50]:
# Test on new, unseen reviews
df2 = pd.read_csv("../data/new_reviews.csv")

reviews = df2['review'].tolist()
true_labels = df2['voted_up'].tolist() 

reviews = [str(text) for text in reviews]
reviews = [re.sub(r'[^\x00-\x7F]+', ' ', text) for text in reviews]

In [54]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Set the model to evaluation mode
model.eval()

batch_size = 2  # Reduced batch size
max_length = 64  # Reduced sequence length

predictions = []

# Process in smaller batches
for i in range(0, len(reviews), batch_size):
    # Tokenize the current batch of reviews
    encodings_chunk = tokenizer(reviews[i:i + batch_size], truncation=True, padding=True, max_length=max_length, return_tensors='pt')

    # Move inputs to the correct device
    input_ids_chunk = encodings_chunk['input_ids'].to(device)
    attention_mask_chunk = encodings_chunk['attention_mask'].to(device)

    # Perform inference with no gradient calculation to save memory
    with torch.no_grad():
        outputs_chunk = model(input_ids=input_ids_chunk, attention_mask=attention_mask_chunk)
        logits_chunk = outputs_chunk.logits

    # Get predicted class (0 = negative, 1 = positive)
    predictions_chunk = torch.argmax(logits_chunk, dim=-1).cpu().numpy()
    predictions.extend(predictions_chunk)


In [55]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)

# Calculate precision, recall, F1-score
precision = precision_score(true_labels, predictions)
recall = recall_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Calculate confusion matrix
conf_matrix = confusion_matrix(true_labels, predictions)

# Print the evaluation metrics
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.96
Precision: 0.97
Recall: 0.99
F1-Score: 0.98
Confusion Matrix:
[[1031  171]
 [  67 5491]]


In [56]:
# Print the first 10 results to verify
label_map = {0: 'Negative', 1: 'Positive'}
predicted_labels = [label_map[pred] for pred in predictions]

for review, sentiment in zip(reviews[:10], predicted_labels[:10]):
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")

Review: If you like long drives on straight roads going past farms, this is the DLC for you. Besides all the farms and small towns, there are some deliveries to be made to gas stations.

Nebraska just doesn't have any real landmarks, but if you go in expecting a farming state, you get that. What there is to map has been mapped and it looks really good.

After playing some more and doing the entire World of Trucks event to explore the state I have some issues though. None of the scenic towns have town markers! I passed through places like Red Cloud and Thedford but they don't have a town sign anywhere, and aren't mentioned on the road shields.

The alignment of some roads is also off. The 6 in Lincoln for example manages to run north-south for a part, whereas IRL it is almost entirely a south-west to north-east road.

I guess some of this may be corrected later but it seems this DLC was not completely finished before release.

Still, if you like ATS, you will probably get this now if no